In [1]:
import qiskit
qiskit.__version__

%load_ext dotenv
%dotenv

# The Hadamard test

> We use a free quantum simulator provided by BlueQubit in order to run our quantum circuits. This is free up to 34 qubits. This is similar to IBM's Aer simulator, which essentially uses multiple classical processors in parallel to emulate quantum hardware with the help of some maths.

While the probability was useful, we can also get JUST the inner product. This is done using the Hadamard test. The initial look of the gate is a lie. It LOOKS like it uses two qubits, one being an ancilla and the other being the operable qubit, which is passed through a controlled unitary gate. But we can select the gate.

$$ U_\psi|0\rangle = |\psi\rangle \Rightarrow \langle 0 | U_\psi^\dagger = \langle \psi | \\
  U_\chi|0\rangle = |\chi\rangle \\
  \Rightarrow \langle \psi | \chi \rangle = \langle 0 | U_\psi^\dagger U_\chi | 0 \rangle \\
  \Rightarrow U = U_\psi^\dagger U_\chi $$

Thus we get the REAL inner product from applying U to |0⟩ once we take the measurement of the ancilla qubit. This is because the probability measurement of the ancilla qubit is the sum of two conjugates. What about the imaginary part? We need to shift the phase of the ancilla qubit by π/2. This is done by adding a $S^\dagger$ gate before the final Hadamard gate.

$$ S^\dagger = \begin{pmatrix} 1 & 0 \\ 0 & -i \end{pmatrix} \\
  \Rightarrow S^\dagger (|0\rangle + |1\rangle) = |0\rangle - i|1\rangle $$

This flips the superposition to give us an imaginary component. Hence when we substract both, we get the $|0\rangle$ measurement. Divide by the total count to give us the value of imaginary part of the inner product.


In [3]:
# imports block
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Operator
import bluequbit
import os

In [4]:
def make_circuit(initial, unitary, flip_to_i):
  quantum_registers = QuantumRegister(3, 'input_q')
  classical_registers = ClassicalRegister(1, 'output_b')
  hadamard_circuit = QuantumCircuit(quantum_registers, classical_registers)

  hadamard_circuit.initialize(initial, [quantum_registers[1], quantum_registers[2]]) # We set up the input state
  hadamard_circuit.h(quantum_registers[0]) # We add a Hadamard gate to the first qubit

  hadamard_circuit.append(Operator(unitary), quantum_registers) # We operate with the unitary

  if flip_to_i: # Flip if asked
    hadamard_circuit.sdg(quantum_registers[0])

  hadamard_circuit.h(quantum_registers[0]) # Final Hadamard gate
  hadamard_circuit.measure(quantum_registers[0], classical_registers[0])

  return hadamard_circuit
  